In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.layers import LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# Q1: Basic Autoencoder
(x_train, _), (x_test, _) = mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

input_img = Input(shape=(784,))
encoded = Dense(32, activation='relu')(input_img)
decoded = Dense(784, activation='sigmoid')(encoded)
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.fit(x_train, x_train, epochs=10, batch_size=256, shuffle=True, validation_data=(x_test, x_test))

# Q2: Denoising Autoencoder
noise_factor = 0.5
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)
x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

autoencoder.fit(x_train_noisy, x_train, epochs=10, batch_size=256, shuffle=True, validation_data=(x_test_noisy, x_test))

# Q3: RNN for Text Generation
text = "Once upon a time in a faraway land, there was a kingdom..."
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts([text])
sequences = tokenizer.texts_to_sequences([text])[0]
padded_sequences = pad_sequences([sequences], maxlen=10)

model = models.Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=10, input_length=10),
    LSTM(50, return_sequences=True),
    LSTM(50),
    Dense(len(tokenizer.word_index) + 1, activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Q4: Sentiment Analysis
max_features = 10000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = pad_sequences(x_train, maxlen=100)
x_test = pad_sequences(x_test, maxlen=100)

model = models.Sequential([
    Embedding(max_features, 128, input_length=100),
    LSTM(64, return_sequences=True),
    LSTM(64),
    Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=5, batch_size=32, validation_split=0.2)

# Confusion Matrix & Report
y_pred = (model.predict(x_test) > 0.5).astype('int32')
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 11s 27ms/step - loss: 0.3799 - val_loss: 0.1914
Epoch 2/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - loss: 0.1815 - val_loss: 0.1549
Epoch 3/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 0.1504 - val_loss: 0.1346
Epoch 4/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: 0.1324 - val_loss: 0.1216
Epoch 5/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 0.1206 - val_loss: 0.1134
Epoch 6/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - loss: 0.1130 - val_loss: 0.1079
Epoch 7/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - loss: 0.1080 - val_loss: 0.1037
Epoch 8/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - loss: 0.1041 - val_loss: 0.1007
Epoch 9/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 0.1014 - val_loss: 0.0981
Epoch 10/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - loss: 0.0988 - val_loss: 0.0964
Epoch 1/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 0.1774 - val_los

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 88s 132ms/step - accuracy: 0.7231 - loss: 0.5244 - val_accuracy: 0.8432 - val_loss: 0.3487
Epoch 2/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 82s 131ms/step - accuracy: 0.8994 - loss: 0.2540 - val_accuracy: 0.8380 - val_loss: 0.3734
Epoch 3/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 86s 138ms/step - accuracy: 0.9302 - loss: 0.1822 - val_accuracy: 0.8398 - val_loss: 0.4271
Epoch 4/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 147s 146ms/step - accuracy: 0.9590 - loss: 0.1179 - val_accuracy: 0.8374 - val_loss: 0.4676
Epoch 5/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 129s 126ms/step - accuracy: 0.9760 - loss: 0.0763 - val_accuracy: 0.8342 - val_loss: 0.5732
782/782 ━━━━━━━━━━━━━━━━━━━━ 30s 37ms/step
[[10688  1812]
 [ 2479 10021]]
              precision    recall  f1-score   support

           0       0.81      0.86      0.83     12500
           1       0.85      0.80      0.82     12500

    accuracy                           0.83     25000
   macro avg       0.83      0.83      0.83     25000
we